### Import bibliotek 

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Definicja narzędzi pomocniczych

In [2]:
def extract(ancestor, selector=None, attribute=None, many=False):
    if selector:
        if many:
            if attribute:
                return [item[attribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        try:
            return ancestor[attribute]
        except TypeError:
            return None
    return ancestor.text.strip()

In [3]:
opinion_schema = {
    "stars": ("span.user-post__score-count",),
    "content": ("div.user-post__text",)
}

### Pobranie z Ceneo.pl opinii o wskazanym produkcie

In [7]:
import os
print(os.listdir("."))


['.gitignore', 'amazon_scraper.ipynb', 'ceneo_scraper.ipynb', 'headears.json', 'opinions', 'sentiment_analysis_en.ipynb']


In [60]:
import json

with open("./headers.json", "r", encoding="utf-8") as jf:
    headers = json.load(jf)


In [61]:
product_id = input("Podaj kod produktu: ")

In [62]:
next_page = f"https://www.ceneo.pl/{product_id}#tab=spec"
response = requests.get(next_page, headers=headers)
if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        brand = extract(page_dom, "table.product-spec__group__attributes > tbody > tr:nth-child(2) > td.product-spec__group__attributes__row__value")
        model = extract(page_dom, "table.product-spec__group__attributes > tbody > tr:nth-child(3) > td.product-spec__group__attributes__row__value")

In [63]:
print(f"Zbieranie opinii dla produktu: {brand} {model}")

Zbieranie opinii dla produktu: Apple iPhone 12‌ Pro 256GB (niebieski)


In [64]:
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
page = 1
while next_page:
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(opinions))
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *value)
                for key, value in opinion_schema.items()
            }
            all_opinions.append(single_opinion)
        try:
            exists = extract(page_dom, "button.pagination__next", "data-hash")
            if exists:
                page+=1
                next_page = f"https://www.ceneo.pl/{product_id}/opinie-{page}"
            else:
                next_page = None
        except TypeError:
            next_page = None    

https://www.ceneo.pl/98018303#tab=reviews
10
https://www.ceneo.pl/98018303/opinie-2
10
https://www.ceneo.pl/98018303/opinie-3
10
https://www.ceneo.pl/98018303/opinie-4
10
https://www.ceneo.pl/98018303/opinie-5
10
https://www.ceneo.pl/98018303/opinie-6
10
https://www.ceneo.pl/98018303/opinie-7
10
https://www.ceneo.pl/98018303/opinie-8
10
https://www.ceneo.pl/98018303/opinie-9
10
https://www.ceneo.pl/98018303/opinie-10
10
https://www.ceneo.pl/98018303/opinie-11
10
https://www.ceneo.pl/98018303/opinie-12
10
https://www.ceneo.pl/98018303/opinie-13
10
https://www.ceneo.pl/98018303/opinie-14
10
https://www.ceneo.pl/98018303/opinie-15
10
https://www.ceneo.pl/98018303/opinie-16
10
https://www.ceneo.pl/98018303/opinie-17
10
https://www.ceneo.pl/98018303/opinie-18
10
https://www.ceneo.pl/98018303/opinie-19
10
https://www.ceneo.pl/98018303/opinie-20
10
https://www.ceneo.pl/98018303/opinie-21
10
https://www.ceneo.pl/98018303/opinie-22
10
https://www.ceneo.pl/98018303/opinie-23
10
https://www.ceneo

### Zapisanie pobranych opinii do plików tekstowych

In [65]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
if not os.path.exists("./opinions/pl"):
    os.mkdir("./opinions/pl")

In [66]:
for idx, single_opinion in enumerate(all_opinions):
    rating = int(round(float(single_opinion["stars"].split("/")[0].replace(",", "."))/5,3)*100)
    sentiment = "P" if rating >= 50 else "N"
    filename = f"./opinions/pl/{brand.replace(' ', '')}_{model.replace(' ', '')}_{rating:03}_{sentiment}_{idx:03}.txt"
    with open(filename, "w", encoding="utf-8") as tf:
        tf.write(single_opinion["content"])